# ДЗ по лекции 2
## Часть 1

>В данной тетрадке используется только датасет "athletes_sochi.txt", наже переменная, в которую забивался путь к датасету

In [86]:
FILEPATH = os.path.join("athletes_sochi.txt")

**Создание модели как в лекции:**

In [4]:
import pandas as pd
import glob
import os
import tqdm
import gc
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
import math
from sklearn import datasets, linear_model, model_selection
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
from tqdm import tqdm

1. Загрузка данных и чистка от Nan

In [89]:
dfs = pd.read_csv(FILEPATH).dropna()

In [90]:
dfs

,age,birthdate,gender,height,name,weight,gold_medals,silver_medals,bronze_medals,total_medals,sport,country
0,17,1996-04-12,Male,1.72,Aaron Blunck,68.0,0,0,0,0,Freestyle Skiing,United States
1,27,1986-05-14,Male,1.85,Aaron March,85.0,0,0,0,0,Snowboard,Italy
2,21,1992-06-30,Male,1.78,Abzal Azhgaliyev,68.0,0,0,0,0,Short Track,Kazakhstan
4,21,1992-07-30,Male,1.86,Adam Barwood,82.0,0,0,0,0,Alpine Skiing,New Zealand
5,21,1992-12-18,Male,1.75,Adam Cieslar,57.0,0,0,0,0,Nordic Combined,Poland
...,...,...,...,...,...,...,...,...,...,...,...,...
2851,28,1985-04-30,Male,1.93,Ziga Pavlin,98.0,0,0,0,0,Ice Hockey,Slovenia
2853,31,1982-12-05,Female,1.70,Zina Kocher,60.0,0,0,0,0,Biathlon,Canada
2854,28,1985-06-14,Female,1.68,Zoe Gillings,65.0,0,0,0,0,Snowboard,Great Britain
2856,22,1991-03-01,Male,1.76,Zongyang Jia,68.0,0,0,1,1,Freestyle Skiing,China


2. Подготовка доп. поля

In [77]:
dfs["BMI"] = dfs["weight"] / (dfs["height"]**2)

In [8]:
myH=1.75
myW=51
myBMI=myW/(myH**2)

In [78]:
alldata = dfs[ ['weight', 'height', 'BMI', 'gender'] ]
alldata

,weight,height,BMI,gender
0,68.0,1.72,22.985398,Male
1,85.0,1.85,24.835646,Male
2,68.0,1.78,21.461937,Male
4,82.0,1.86,23.702162,Male
5,57.0,1.75,18.612245,Male
...,...,...,...,...
2851,98.0,1.93,26.309431,Male
2853,60.0,1.70,20.761246,Female
2854,65.0,1.68,23.030045,Female
2856,68.0,1.76,21.952479,Male


3. Подготовка входных данных:
* набор пар для входных векторов X: пары это вес и рост
* набор пар для выходных векторов Y: это значения 1 -- male, 0 -- female
* перемешать набор данных весь и взять 20% отложить как test, остальное будет train

In [41]:
alldata = alldata.sample(frac=1)

In [11]:
alldata

,weight,height,BMI,gender
2681,65.0,1.68,23.030045,Female
2022,62.0,1.77,19.789971,Male
234,64.0,1.71,21.887076,Female
1788,68.0,1.73,22.720438,Male
1518,90.0,1.80,27.777778,Male
...,...,...,...,...
363,66.0,1.74,21.799445,Female
395,82.0,1.94,21.787650,Male
1127,74.0,1.80,22.839506,Male
1724,55.0,1.65,20.202020,Female


In [42]:
allinput = alldata[ ['weight', 'height'] ]
alloutput = alldata["gender"].apply(lambda x: int(x == "Male"))

In [13]:
allinput

,weight,height
2681,65.0,1.68
2022,62.0,1.77
234,64.0,1.71
1788,68.0,1.73
1518,90.0,1.80
...,...,...
363,66.0,1.74
395,82.0,1.94
1127,74.0,1.80
1724,55.0,1.65


In [14]:
alloutput

2681    0
2022    1
234     0
1788    1
1518    1
       ..
363     0
395     1
1127    1
1724    0
839     1
Name: gender, Length: 2479, dtype: int64

4. Разделение выборок на **обучающую** (X_train, Y_train) и **тестовую** (X_test, Y_test)

In [15]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(allinput, alloutput, test_size=0.2)

5. Создание модели Логистической Регрессии

In [16]:
model = linear_model.LogisticRegression()

6. Обучение модели на обучающей выборке

In [17]:
model.fit(X_train, Y_train)

LogisticRegression()

7. Коэффициенты модели

In [18]:
model.coef_

array([[0.16458263, 6.91166173]])

8. Предсказание пола с помощью выборки X_test (тестовая часть выборки по полу)

In [19]:
Y_test_predicted = model.predict(X_test)

In [20]:
mysex=model.predict([[myW, myH]])

In [21]:
mysex

array([0], dtype=int64)

In [22]:
Y_test_predicted

array([1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,

9. Оценка качества алгоритма

In [23]:
[tn, fp], [fn, tp] = confusion_matrix(Y_test, Y_test_predicted)
confusion_matrix(Y_test, Y_test_predicted)

array([[152,  24],
       [ 36, 284]], dtype=int64)

In [24]:
precision = tp / (tp + fp)
recall = tp / tp + fn

print("Precision: ", tp / (tp + fp))
print("Recall: ", tp / (tp + fn))
print("Accuracy: ", (tn + tp) / (tp + fp + tn + fn))
print("F1: ",  (precision * recall) / ((precision) + recall))

Precision:  0.922077922077922
Recall:  0.8875
Accuracy:  0.8790322580645161
F1:  0.8996575342465752


In [25]:
from sklearn.metrics import classification_report

In [26]:
report = classification_report(Y_test, model.predict(X_test), target_names=['Male', 'Female'])
print(report)

              precision    recall  f1-score   support

        Male       0.81      0.86      0.84       176
      Female       0.92      0.89      0.90       320

    accuracy                           0.88       496
   macro avg       0.87      0.88      0.87       496
weighted avg       0.88      0.88      0.88       496



## ДЗ 1 Классификация Домашнее задание из двух частей.

### Креативная часть: 
Найти парамеры и комбинации их, которые будут увеличивать accuracy (для модели с лекции, которая выше)

In [27]:
alldata

,weight,height,BMI,gender
2681,65.0,1.68,23.030045,Female
2022,62.0,1.77,19.789971,Male
234,64.0,1.71,21.887076,Female
1788,68.0,1.73,22.720438,Male
1518,90.0,1.80,27.777778,Male
...,...,...,...,...
363,66.0,1.74,21.799445,Female
395,82.0,1.94,21.787650,Male
1127,74.0,1.80,22.839506,Male
1724,55.0,1.65,20.202020,Female


При введении параметра индекса массы тела 'BMI', accuracy увеличивается почти на 5%

In [28]:
allinput_2 = alldata[ ['weight', 'height', 'BMI'] ]
alloutput_2 = alldata["gender"].apply(lambda x: int(x == "Male"))

X_train_2, X_test_2, Y_train_2, Y_test_2 = model_selection.train_test_split(allinput_2, alloutput_2, test_size=0.2)

model.fit(X_train_2, Y_train_2)

Y_test_2_predicted = model.predict(X_test_2)

[tn, fp], [fn, tp] = confusion_matrix(Y_test_2, Y_test_2_predicted)
confusion_matrix(Y_test_2, Y_test_2_predicted)

array([[158,  41],
       [ 24, 273]], dtype=int64)

In [29]:
precision = tp / (tp + fp)
recall = tp / tp + fn

print("Precision: ", tp / (tp + fp))
print("Recall: ", tp / (tp + fn))
print("Accuracy: ", (tn + tp) / (tp + fp + tn + fn))
print("F1: ",  2 * (precision * recall) / ((precision) + recall))

Precision:  0.8694267515923567
Recall:  0.9191919191919192
Accuracy:  0.8689516129032258
F1:  1.6804136402806846


Добавление Возраста 'age' в список параметров не дало должного результата. По отношению к предыдущей попытке повышения accuracy результат ухудшился, но относительно первого построения модели стал лучше.

In [30]:
alldata = dfs[ ['weight', 'height', 'BMI', 'age', 'gender'] ]
allinput_3 = alldata[ ['weight', 'height', 'BMI', 'age'] ]
alloutput_3 = alldata["gender"].apply(lambda x: int(x == "Male"))

X_train_3, X_test_3, Y_train_3, Y_test_3 = model_selection.train_test_split(allinput_3, alloutput_3, test_size=0.2)

model.fit(X_train_3, Y_train_3)

Y_test_3_predicted = model.predict(X_test_3)

[tn, fp], [fn, tp] = confusion_matrix(Y_test_3, Y_test_3_predicted)
confusion_matrix(Y_test_3, Y_test_3_predicted)

array([[159,  36],
       [ 38, 263]], dtype=int64)

In [31]:
precision = tp / (tp + fp)
recall = tp / tp + fn

print("Precision: ", tp / (tp + fp))
print("Recall: ", tp / (tp + fn))
print("Accuracy: ", (tn + tp) / (tp + fp + tn + fn))
print("F1: ",  2 * (precision * recall) / ((precision) + recall))

Precision:  0.8795986622073578
Recall:  0.8737541528239202
Accuracy:  0.8508064516129032
F1:  1.7203958403220394


>F-мера - среднее гармоническое precision и recall; критерий качества, объединяющий параметры precision и recall (точность и полнота)

Расчет F-меры для двух вариантов с добавочными критериями показал, однако, что стоит отдать предпочтение второму варианту алгоритма.

### Техническая часть: 
Сделать под это всё удобные обёртки.  
Например, это функция, которая просто принимает список фичей, а на выходе показывает результат фита

1. Функция data_cooking() выделяет в исходном (предобработанном) датасете указанные пользователем столбцы факторов и целевой столбец (y)  
Удобно для отработки гипотез, переиспользуемый код, не зависящий от конкретного датасета

In [79]:
# функция предназначена для подготовки тренировочной и тестовой выборок по набору факторов
# принимает на вход подготовленный датасет, список столбцов-факторов, целевой столбец


def data_cooking (datafr, factors, y):
    input_data = datafr[ factors ]
    output_data = datafr[y]
    x_train, x_test, y_train, y_test = model_selection.train_test_split(input_data, output_data, test_size=0.2)
    return x_train, x_test, y_train, y_test


# доработка датасета
dfs["gender"] = dfs["gender"].apply(lambda x: int(x == "Male"))


# пример применения функции
my_factors = ['weight', 'height', 'BMI']
my_y = 'gender'
x_train, x_test, y_train, y_test = data_cooking(dfs, my_factors, my_y)
print(x_train)
print()
print(x_test)
print()
print(y_train)
print()
print(y_test)

      weight  height        BMI
1257    95.0    1.80  29.320988
442     66.0    1.70  22.837370
367     71.0    1.80  21.913580
2508    63.0    1.70  21.799308
2453    60.0    1.64  22.308150
...      ...     ...        ...
1633    72.0    1.80  22.222222
1390    64.0    1.74  21.138856
1040   101.0    1.91  27.685645
2278    72.0    1.70  24.913495
289     77.0    1.87  22.019503

[1983 rows x 3 columns]

      weight  height        BMI
2087    82.0    1.79  25.592210
678     80.0    1.80  24.691358
368     95.0    2.00  23.750000
2803    52.0    1.57  21.096191
2523    64.0    1.70  22.145329
...      ...     ...        ...
495     54.0    1.68  19.132653
1521    65.0    1.75  21.224490
2775    57.0    1.69  19.957284
1327    55.0    1.63  20.700817
1820    85.0    1.81  25.945484

[496 rows x 3 columns]

1257    1
442     1
367     1
2508    0
2453    0
       ..
1633    0
1390    0
1040    1
2278    1
289     1
Name: gender, Length: 1983, dtype: int64

2087    1
678     1
368     1

2. Простая функция train_LogisticRegression() немного оптимизирует процесс отработки гипотез и возвращает уже натренированную модель

In [82]:
# функция тренирует модель логистической регрессии и делает более удобный процесс введения в модель новых факторов
# функция принимает на вход тренировочную выборку (x и y)


def train_LogisticRegression (x_train, y_train):
    new_model = linear_model.LogisticRegression()
    new_model.fit(x_train, y_train)
    return new_model


# пример применения функции
my_model = train_LogisticRegression(x_train, y_train)

3. Функция conf_matrix() принимает на вход тестовую выборку (х и у) и натренированную модель; предназначена для автоматизированного расчета параметров оценки качества модели

In [84]:
# функция предназначена для автоматизирования процесса рассчета матрицы ошибок
# на вход передаются тестовые выборки X и Y, натренированная модель

def conf_matrix (x_test, y_test, model):
    y_predicted = model.predict(x_test)
    [tn, fp], [fn, tp] = confusion_matrix(y_test, y_predicted)
    print('Confusion matrix:')
    print(confusion_matrix(y_test, y_predicted))
    print()

    precision = tp / (tp + fp)
    recall = tp / tp + fn
    print("Precision: ", tp / (tp + fp))
    print("Recall: ", tp / (tp + fn))
    print("Accuracy: ", (tn + tp) / (tp + fp + tn + fn))
    print("F1: ",  2 * (precision * recall) / ((precision) + recall))
    
    
# пример применения функции
conf_matrix(x_test, y_test, my_model)

Confusion matrix:
[[169  39]
 [ 24 264]]

Precision:  0.8712871287128713
Recall:  0.9166666666666666
Accuracy:  0.8729838709677419
F1:  1.6838882510524302


## ДЗ 2 Кластеризация

Часть домашнего задания с кластеризацией находится в файле "HW_2_Part_2"